# TransitKit Quick Start Guide

This notebook demonstrates the basic workflow for transit detection.

**Topics:** Synthetic signals, noise, BLS detection, visualization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from transitkit.core import (
    generate_transit_signal_mandel_agol,
    find_transits_bls_advanced,
    add_noise,
    TransitParameters
)
print('TransitKit imported!')

## 1. Generate Synthetic Transit

In [ ]:
# True parameters
TRUE_PERIOD = 5.0
TRUE_T0 = 2.5
TRUE_DEPTH = 0.01  # 1%

# Generate data
time = np.linspace(0, 30, 2000)
flux = generate_transit_signal_mandel_agol(time, period=TRUE_PERIOD, t0=TRUE_T0, depth=TRUE_DEPTH)
flux_noisy = add_noise(flux, noise_level=0.001, seed=42)

print(f'Generated {len(time)} points, {int(30/TRUE_PERIOD)} transits')

In [ ]:
# Plot light curve
plt.figure(figsize=(12, 4))
plt.plot(time, flux_noisy, 'k.', ms=1, alpha=0.5)
plt.xlabel('Time (days)')
plt.ylabel('Normalized Flux')
plt.title('Synthetic Light Curve with Transits')
plt.show()

## 2. Detect Transit with BLS

In [ ]:
result = find_transits_bls_advanced(time, flux_noisy, min_period=1.0, max_period=15.0)

print('BLS Results:')
print(f"  Period: {result['period']:.4f} d (true: {TRUE_PERIOD})")
print(f"  Depth:  {result['depth']*1e6:.0f} ppm (true: {TRUE_DEPTH*1e6:.0f})")
print(f"  SNR:    {result['snr']:.1f}")

In [ ]:
# Plot periodogram
plt.figure(figsize=(10, 4))
plt.plot(result['all_periods'], result['all_powers'], 'k-', lw=0.5)
plt.axvline(result['period'], color='r', ls='--', label=f"Detected: {result['period']:.3f} d")
plt.xlabel('Period (days)')
plt.ylabel('BLS Power')
plt.title('BLS Periodogram')
plt.legend()
plt.xscale('log')
plt.show()

## 3. Phase-Folded Plot

In [ ]:
# Phase fold
phase = ((time - result['t0']) / result['period']) % 1
phase[phase > 0.5] -= 1

plt.figure(figsize=(8, 5))
plt.plot(phase, flux_noisy, 'k.', ms=2, alpha=0.3)
plt.xlabel('Phase')
plt.ylabel('Flux')
plt.title(f"Phase-Folded (P={result['period']:.4f} d)")
plt.xlim(-0.1, 0.1)
plt.show()

## Summary

We successfully:
1. Generated synthetic transit
2. Detected it with BLS
3. Recovered correct period

**Next:** See other notebooks for TESS data, multi-method detection, and TTV analysis.